<h3><center>Uber Data Analysis - April 2023</center></h3>

#### importing necessary packages and data file

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [2]:
trips = pq.read_table("yellow_tripdata_2023-04.parquet")
trips = trips.to_pandas()

In [3]:
trips

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,28.90,3.5,0.5,6.00,0.00,1.0,39.90,2.5,0.00
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,70.00,0.0,0.5,0.00,6.55,1.0,81.80,2.5,1.25
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,11.40,3.5,0.5,2.00,0.00,1.0,18.40,2.5,0.00
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,10.00,3.5,0.5,1.00,0.00,1.0,16.00,2.5,0.00
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,11.40,1.0,0.5,1.00,0.00,1.0,17.40,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288245,2,2023-04-30 23:50:19,2023-05-01 00:06:11,NaN,2.02,NaN,None,246,79,0,11.41,0.0,0.5,3.08,0.00,1.0,18.49,NaN,NaN
3288246,2,2023-04-30 23:28:32,2023-04-30 23:38:32,NaN,2.36,NaN,None,114,68,0,13.19,0.0,0.5,0.00,0.00,1.0,17.19,NaN,NaN
3288247,2,2023-04-30 23:25:26,2023-04-30 23:44:16,NaN,4.64,NaN,None,137,74,0,17.90,0.0,0.5,0.00,0.00,1.0,21.90,NaN,NaN
3288248,1,2023-04-30 23:11:59,2023-04-30 23:26:19,NaN,0.00,NaN,None,162,151,0,16.00,1.0,0.5,4.20,0.00,1.0,27.20,NaN,NaN


#### Split above Dataframe into fact and dim tables as per data model

In [4]:
trips['tpep_pickup_datetime'] = pd.to_datetime(trips['tpep_pickup_datetime'])
trips['tpep_dropoff_datetime'] = pd.to_datetime(trips['tpep_dropoff_datetime'])

In [5]:
trips = trips.drop_duplicates().reset_index(drop=True)
trips['trip_id'] = trips.index
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,28.9,3.5,0.5,6.0,0.00,1.0,39.9,2.5,0.00,0
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,70.0,0.0,0.5,0.0,6.55,1.0,81.8,2.5,1.25,1
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,11.4,3.5,0.5,2.0,0.00,1.0,18.4,2.5,0.00,2
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,10.0,3.5,0.5,1.0,0.00,1.0,16.0,2.5,0.00,3
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,11.4,1.0,0.5,1.0,0.00,1.0,17.4,2.5,0.00,4


##### Keeping only 2023 April data

In [19]:
trips['month'] = trips['tpep_pickup_datetime'].dt.month
trips['year'] = trips['tpep_pickup_datetime'].dt.year
trips = trips[trips['year']==2023]
trips = trips[trips['month']==4]
len(trips)

3288155

In [20]:
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id,month,year
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,...,0.5,6.0,0.00,1.0,39.9,2.5,0.00,0,4,2023
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,...,0.5,0.0,6.55,1.0,81.8,2.5,1.25,1,4,2023
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,...,0.5,2.0,0.00,1.0,18.4,2.5,0.00,2,4,2023
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,...,0.5,1.0,0.00,1.0,16.0,2.5,0.00,3,4,2023
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,...,0.5,1.0,0.00,1.0,17.4,2.5,0.00,4,4,2023


##### Datetime_dim

In [21]:
datetime_dim = trips[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


In [22]:
datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
datetime_dim

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2023-04-01 00:14:49,0,1,4,2023,5,2023-04-01 00:45:01,0,1,4,2023,5
1,1,2023-04-01 00:00:24,0,1,4,2023,5,2023-04-01 00:56:19,0,1,4,2023,5
2,2,2023-04-01 00:03:50,0,1,4,2023,5,2023-04-01 00:14:42,0,1,4,2023,5
3,3,2023-04-01 00:53:18,0,1,4,2023,5,2023-04-01 01:01:28,1,1,4,2023,5
4,4,2023-04-01 00:07:00,0,1,4,2023,5,2023-04-01 00:17:16,0,1,4,2023,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288150,3288150,2023-04-30 23:50:19,23,30,4,2023,6,2023-05-01 00:06:11,0,1,5,2023,0
3288151,3288151,2023-04-30 23:28:32,23,30,4,2023,6,2023-04-30 23:38:32,23,30,4,2023,6
3288152,3288152,2023-04-30 23:25:26,23,30,4,2023,6,2023-04-30 23:44:16,23,30,4,2023,6
3288153,3288153,2023-04-30 23:11:59,23,30,4,2023,6,2023-04-30 23:26:19,23,30,4,2023,6


##### Passenger count and Trip distance

In [23]:
passenger_count_dim = trips[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]

trip_distance_dim = trips[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]
trip_distance_dim

,trip_distance_id,trip_distance
0,0,4.90
1,1,21.89
2,2,1.30
3,3,1.50
4,4,1.49
...,...,...
3288150,3288150,2.02
3288151,3288151,2.36
3288152,3288152,4.64
3288153,3288153,0.00


##### PULocationID and DOLocationID

In [24]:
PULocationID_dim = trips[['PULocationID']].reset_index(drop=True)
PULocationID_dim['PULocationID_id'] = PULocationID_dim.index
PULocationID_dim = PULocationID_dim[['PULocationID_id', 'PULocationID']]

DOLocationID_dim = trips[['DOLocationID']].reset_index(drop=True)
DOLocationID_dim['DOLocationID_id'] = DOLocationID_dim.index
DOLocationID_dim = DOLocationID_dim[['DOLocationID_id', 'DOLocationID']]


##### Ratecode and Payment type

In [25]:
rate_dict = {1:'Standard rate',
            2:'JFK',
            3:'Newark',
            4:'Nessau or Westchester',
            5:'Negotiated fare',
            6:'Group ride'}

RatecodeID_dim = trips[['RatecodeID']].reset_index(drop=True)
RatecodeID_dim['RatecodeID_id'] = RatecodeID_dim.index
RatecodeID_dim['Ratecode_name'] = RatecodeID_dim['RatecodeID'].map(rate_dict)
RatecodeID_dim = RatecodeID_dim[['RatecodeID_id', 'RatecodeID', 'Ratecode_name']]
RatecodeID_dim

,RatecodeID_id,RatecodeID,Ratecode_name
0,0,1.0,Standard rate
1,1,2.0,JFK
2,2,1.0,Standard rate
3,3,1.0,Standard rate
4,4,1.0,Standard rate
...,...,...,...
3288150,3288150,NaN,NaN
3288151,3288151,NaN,NaN
3288152,3288152,NaN,NaN
3288153,3288153,NaN,NaN


In [26]:
pay_dict = {1:'Credit card',
            2:'Cash',
            3:'No charge',
            4:'Dispute',
            5:'Unknown',
            6:'Voided trip'}

payment_type_dim = trips[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_name'] = payment_type_dim['payment_type'].map(pay_dict)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_name']]
payment_type_dim

,payment_type_id,payment_type,payment_name
0,0,1,Credit card
1,1,2,Cash
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card
...,...,...,...
3288150,3288150,0,NaN
3288151,3288151,0,NaN
3288152,3288152,0,NaN
3288153,3288153,0,NaN


##### Fact table

In [27]:
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id,month,year
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,...,0.5,6.0,0.00,1.0,39.9,2.5,0.00,0,4,2023
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,...,0.5,0.0,6.55,1.0,81.8,2.5,1.25,1,4,2023
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,...,0.5,2.0,0.00,1.0,18.4,2.5,0.00,2,4,2023
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,...,0.5,1.0,0.00,1.0,16.0,2.5,0.00,3,4,2023
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,...,0.5,1.0,0.00,1.0,17.4,2.5,0.00,4,4,2023


In [28]:
fact_table = trips.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(RatecodeID_dim, left_on='trip_id', right_on='RatecodeID_id') \
             .merge(PULocationID_dim, left_on='trip_id', right_on='PULocationID_id') \
             .merge(DOLocationID_dim, left_on='trip_id', right_on='DOLocationID_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'RatecodeID_id', 'store_and_fwd_flag', 'PULocationID_id', 'DOLocationID_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']]

In [29]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,RatecodeID_id,store_and_fwd_flag,PULocationID_id,DOLocationID_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,1,0,0,0,0,N,0,0,0,28.90,3.5,0.5,6.00,0.00,1.0,39.90,2.5,0.00
1,1,2,1,1,1,1,N,1,1,1,70.00,0.0,0.5,0.00,6.55,1.0,81.80,2.5,1.25
2,2,1,2,2,2,2,N,2,2,2,11.40,3.5,0.5,2.00,0.00,1.0,18.40,2.5,0.00
3,3,1,3,3,3,3,N,3,3,3,10.00,3.5,0.5,1.00,0.00,1.0,16.00,2.5,0.00
4,4,2,4,4,4,4,N,4,4,4,11.40,1.0,0.5,1.00,0.00,1.0,17.40,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288055,3288150,2,3288150,3288150,3288150,3288150,None,3288150,3288150,3288150,43.11,0.0,0.5,9.42,0.00,1.0,56.53,NaN,NaN
3288056,3288151,2,3288151,3288151,3288151,3288151,None,3288151,3288151,3288151,30.22,0.0,0.5,5.00,0.00,1.0,39.22,NaN,NaN
3288057,3288152,2,3288152,3288152,3288152,3288152,None,3288152,3288152,3288152,10.51,0.0,0.5,0.00,0.00,1.0,14.51,NaN,NaN
3288058,3288153,2,3288153,3288153,3288153,3288153,None,3288153,3288153,3288153,31.43,0.0,0.5,0.00,0.00,1.0,35.43,NaN,NaN


#### Saving fact and dim tables as Parquet files

In [30]:
fact_table.to_parquet("Parquet files/fact_table.parquet", index=False)
datetime_dim.to_parquet("Parquet files/datetime_dim.parquet", index=False)
PULocationID_dim.to_parquet("Parquet files/PULocationID_dim.parquet", index=False)
DOLocationID_dim.to_parquet('Parquet files/DOLocationID_dim.parquet', index=False)
passenger_count_dim.to_parquet('Parquet files/passenger_count_dim.parquet', index=False)
trip_distance_dim.to_parquet("Parquet files/trip_distance_dim.parquet", index=False)
RatecodeID_dim.to_parquet("Parquet files/RatecodeID_dim.parquet", index=False)
payment_type_dim.to_parquet("Parquet files/payment_type_dim.parquet", index=False)


##### Addng Location name column

In [2]:
pick_loc = pq.read_table("Parquet files/PULocationID_dim.parquet")
pick_loc = pick_loc.to_pandas()
drop_loc = pq.read_table("Parquet files/DOLocationID_dim.parquet")
drop_loc = drop_loc.to_pandas()

In [3]:
pick_loc

,PULocationID_id,PULocationID
0,0,48
1,1,132
2,2,148
3,3,249
4,4,158
...,...,...
3288150,3288150,246
3288151,3288151,114
3288152,3288152,137
3288153,3288153,162


In [5]:
taxi_zone = pd.read_csv("taxi_zone_lookup.csv")
taxi_zone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [6]:
pick_loc = pick_loc.merge(taxi_zone, left_on='PULocationID', right_on='LocationID')
drop_loc = drop_loc.merge(taxi_zone, left_on='DOLocationID', right_on='LocationID')

In [7]:
pick_loc.drop(columns = 'LocationID', inplace=True)
drop_loc.drop(columns = 'LocationID', inplace=True)

In [8]:
drop_loc['Borough'].unique()

array(['Queens', 'Manhattan', 'Brooklyn', 'Unknown', 'Bronx',
       'Staten Island', 'EWR'], dtype=object)

In [9]:
pick_loc.to_parquet("Pick-up_loc.parquet")
drop_loc.to_parquet("Drop_loc.parquet")